# Deep Convolutional Generative Adversarial Network

<img src="https://miro.medium.com/max/4124/1*KvMnRfb76DponICrHIbSdg.png"  width="800" height="400">

##### Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from time import time
import os
import pandas as pd
os.chdir('C:/Users/Nicolas/Documents/Data/Faces')
import argparse
import math
import re
import itertools
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

##### Function to order images

In [2]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

##### Loading all file names

In [3]:
files = sorted_alphanumeric(glob('combined/*.jpg'))

##### Loading the 800 hundred women pictures

In [4]:
def load_women():
    faces = pd.read_csv('800_women.csv', header=None).values
    faces = faces.ravel().tolist()
    return faces
faces = load_women()
y = np.repeat(1, len(faces))

#### This is the shape width/height

In [5]:
dim = 60

#### Cropping function

In [6]:
def crop(img):
    if img.shape[0]<img.shape[1]:
        x = img.shape[0]
        y = img.shape[1]
        crop_img = img[: , int(y/2-x/2):int(y/2+x/2)]
    else:
        x = img.shape[1]
        y = img.shape[0]
        crop_img = img[int(y/2-x/2):int(y/2+x/2) , :]

    return crop_img

##### Loading and cropping images

In [7]:
print('Scaling...', end='')
start = time()
x = []
num_to_load = len(faces)  
for ix, file in enumerate(faces[:num_to_load]): 
    image = plt.imread(file, 'jpg')
    image = Image.fromarray(image).resize((dim, dim)).convert('L')
    image = crop(np.array(image))
    x.append(image)
print(f'\rDone. {int(time() - start)} seconds')

Done. 0 seconds


##### Turning the pictures into arrays

In [8]:
x = np.divide(np.array(x, dtype=np.float32), 255).reshape(-1, 1, 60, 60)
y = np.array(y, dtype=np.float32)

##### Turning the targets into a 2D matrix

In [9]:
assert x.ndim == 4, 'The input is the wrong shape!'

In [10]:
yy, xx = y.nbytes, x.nbytes

In [11]:
print(f'The size of X is {xx:,} bytes and the size of Y is {yy:,} bytes.')

The size of X is 11,520,000 bytes and the size of Y is 3,200 bytes.


In [12]:
files, faces = None, None

In [13]:
print('Scaling...', end='')
image_size = x.shape[1] * x.shape[1]
print('\rDone.     ')

Done.     


##### Sending the tensors to CUDA

In [14]:
if torch.cuda.is_available():
    x = torch.from_numpy(x) 
    y = torch.from_numpy(y)
    print('Tensors successfully flushed to CUDA.')
else:
    print('CUDA not available!')

Tensors successfully flushed to CUDA.


##### Making a dataset class

In [15]:
class Face():
    
    def __init__(self):
        self.len = x.shape[0]
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return x[index], y[index].unsqueeze(0) 
    
    def __len__(self):
        return self.len

##### Instantiating the class

In [16]:
data = Face()

##### Parsing the args

In [17]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=5_000, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=32, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.001, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.3, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=128, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=60, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=5, help="interval between image sampling")
opt, unknown = parser.parse_known_args()
print(opt)

Namespace(b1=0.3, b2=0.999, batch_size=32, channels=1, img_size=60, latent_dim=128, lr=0.001, n_cpu=8, n_epochs=5000, sample_interval=5)


In [18]:
cuda = True if torch.cuda.is_available() else False

##### Initializing the weights

In [19]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

##### Creating the generator

In [20]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = opt.img_size // 4 ## 15
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2)) # (100, 128*15^2) 28800
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

##### Creating the discriminator

In [21]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // (2 ** 4) # gives 3
        self.adv_layer = nn.Sequential(nn.Linear(128 * 4 ** 2, 1), nn.Sigmoid()) # s'attend à (1152, 1)

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1) # torch.Size([64, 2048])
        validity = self.adv_layer(out)

        return validity

##### Creating the loss functions

In [22]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

##### Loading the trained models

In [23]:
generator.load_state_dict(torch.load('deep_conv_gan_generator'))
discriminator.load_state_dict(torch.load('deep_conv_gan_discriminator'))

<All keys matched successfully>

In [24]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [25]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (adv_laye

##### Configure the `dataloader`

In [26]:
# Configure data loader
dataloader = torch.utils.data.DataLoader(data,
    batch_size=opt.batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

##### Train the model

In [27]:
for epoch in range(1, opt.n_epochs + 1):
    for i, (imgs, _) in enumerate(dataloader):
        
        break ##### DO NOT TRAIN THE MODEL AGAIN
        
        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        
        if not os.path.isdir('dcgan_800_women'):
            os.mkdir('dcgan_800_women')
        
    batches_done = epoch * len(dataloader) + i + 1
    
    if epoch >= 500 and epoch % 100 == 0:
        val = input("\nContinue training? [y/n]: ")
        print()
        if val in ('y', 'yes'):
            val = True
            pass
        elif val in ('n', 'no'):
            break  
        else:
            pass
        
    if batches_done % opt.sample_interval == 0:
        save_image(gen_imgs.data[:25], "dcgan/%d.png" % batches_done, nrow=5, normalize=True)

    if epoch % 50 == 0:
        print(
            "[Epoch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, d_loss.item(), g_loss.item())
        )

[Epoch 50/5000] [D loss: 0.730409] [G loss: 0.706794]
[Epoch 100/5000] [D loss: 0.573488] [G loss: 0.627583]
[Epoch 150/5000] [D loss: 0.654021] [G loss: 0.734065]
[Epoch 200/5000] [D loss: 0.706726] [G loss: 0.803335]
[Epoch 250/5000] [D loss: 0.706729] [G loss: 0.742541]
[Epoch 300/5000] [D loss: 0.675414] [G loss: 0.822383]
[Epoch 350/5000] [D loss: 0.700076] [G loss: 0.858581]
[Epoch 400/5000] [D loss: 0.785222] [G loss: 0.834059]
[Epoch 450/5000] [D loss: 0.621714] [G loss: 0.968932]



Continue training? [y/n]:  n


##### Save the model

In [31]:
torch.save(generator.state_dict(), 'deep_conv_gan_generator')
torch.save(discriminator.state_dict(), 'deep_conv_gan_discriminator')

##### Create a function to save images

In [32]:
def sample_image(n_row, batches_done):
    z = Variable(Tensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    gen_imgs = generator(z)
    save_image(gen_imgs.data, "dcgan_800_women/%d.png" % batches_done, nrow=n_row, normalize=True)

##### Generate 25,000 pictures

In [33]:
images = 0

for epoch in range(1, 2_50 + 1): 
    for i, (imgs, _) in enumerate(dataloader, 1):
        
        with torch.no_grad():

            # Adversarial ground truths
            valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
            fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

            # Configure input
            real_imgs = Variable(imgs.type(Tensor))

            batches_done = epoch * len(dataloader) + i
            sample_image(n_row=5, batches_done=batches_done)
            images += 25
            
    if images % 5_000 == 0:
        print(f'Pictures created: {images:,}')
        
    if len(os.listdir(os.path.join(os.getcwd(), 'dcgan_800_women')))  >= 1_000:
        print('\n25,000 images successfully generated.')
        break

Pictures created: 5,000
Pictures created: 10,000
Pictures created: 15,000
Pictures created: 20,000
Pictures created: 25,000

25,000 images successfully generated.
